LangChain의 RAG 컴포넌트 - 벡터 저장소(Vector Store)

- 학습 목표
	- 임베딩 모델과 벡터 데이터베이스를 효과적으로 연동할 수 있다.
---
### Vector Store ( 벡터 저장소 )
- 개념
	- 백터화된 데이터를 효율적으로 저장하고 검색하기 위한 특수 데이터베이스 시스템
	- 텍스트나 이미지 등의 비정형 데이터를 고차원 벡터 공간에 매핑하여 저장
	- 유사도 기반 검색을 통해 의미적으로 가까운 데이터를 빠르게 검색 가능
- LangChain의 벡터 저장소 종류
	- Chroma : 경량화된 임베딩 데이터베이스로 로컬 개발에 적합
	- FAISS : Facebook AI가 개발한 고성능 유사도 검색 라이브러리
	- Pinecone : 완전 관리형 벡터 데이터베이스 서비스
	- Milvus : 분산 벡터 데이터베이스로 대규모 데이터 처리에 적합
	- PostgreSQL : pgvector확장을 통해 벡터 저장 및 검색 기능을 제공
- 주요 기능
	- 백터 색인화 : 효율적인 검색을 위한 데이터 구조화를 수행
	- 근접 이웃 검색 : 주어진 쿼리와 가장 유사한 벡터들을 검색
	- 메타데이터 관리 : 벡터와 관련된 부가 정보를 함께 저장하고 검색
- 사용 사례
	- 시멘틱 문서 검색 : 문서의 의미를 이해하여 검색
	- 추천 시스템 : 유사한 아이템을 추천
	- 중복 데이터 감지 : 유사한 콘텐츠를 검색
	- 질의응답 시스템 : 관련 문서에서 답변을 생성하는데 필요한 근거를 검색
- 벡터 저장소 선택 가이드
	- Chroma
		- 장점 : 설정간단, 로컬 개발 용이
		- 단점 : 대규모 데이터 처리 제한적
		- 추천 사용 사례 : 프로토타입, 소규모 프로젝트
	- FAISS
		- 장점 : 빠른 검새 소도, 대용량 처리
		- 단점 : 별도 저장소 없음 (메모리, 파일만)
		- 추천 사용 사례 : 고성능 검색, 대규모 데이터
	- Pinecone
		- 장점 : 완전 관리형, 확장성 우수
		- 단점 : 유료 서비스, API 키 필요
		- 추천 사용 사례 : 프로덕션 환경, 클라우드 배포

### 1. Chroma
	- 사용자 편의성이 우수한 오픈소스 벡터 저장소
	- langchain-chroma 패키지 설치

(1) 백터 저장소 초기화

In [18]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

qwen3-embedding:0.6b 모델 사용시 다운 필요
- ollama pull qwen3-embedding:0.6b

In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(
	model="qwen3-embedding:0.6b"
)

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# 벡터 저장소 생성
from langchain_chroma import Chroma

chroma_db = Chroma(
	collection_name="ai_sample_collection",
	embedding_function=embeddings_model,
	persist_directory="../chroma_db"
)

# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_5', 'DOC_3'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.',
  '머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.',
  '딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'AI 개론'},
  {'source': 'AI 개론'},
  {'source': '딥러닝 입문'},
  {'source': '딥러닝 입문'}]}

(2) 벡터 저장소 관리
	- 문서 추가: vector_store.add_documents(documents, ids)

In [20]:
from langchain_core.documents import Document

# 문서 데이터 - (텍스트, 소스)
documents = [
    ("인공지능은 컴퓨터 과학의 한 분야입니다.", "AI 개론"),
    ("머신러닝은 인공지능의 하위 분야입니다.", "AI 개론"),
    ("딥러닝은 머신러닝의 한 종류입니다.", "딥러닝 입문"),
    ("자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.", "AI 개론"),
    ("컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.", "딥러닝 입문")
]

# Document 객체 생성
doc_objects = []

for content, source in documents:
	doc = Document(
		page_content=content,
		metadata={"source": source}
	)
	doc_objects.append(doc)

# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = chroma_db.add_documents(documents=doc_objects, ids = doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print("document 객체 : ", doc_objects)
print(f"벡터 저장소에 성공적으로 추가된 문서 개수 : {len(added_doc_ids)}")
print(added_doc_ids)

document 객체 :  [Document(metadata={'source': 'AI 개론'}, page_content='인공지능은 컴퓨터 과학의 한 분야입니다.'), Document(metadata={'source': 'AI 개론'}, page_content='머신러닝은 인공지능의 하위 분야입니다.'), Document(metadata={'source': '딥러닝 입문'}, page_content='딥러닝은 머신러닝의 한 종류입니다.'), Document(metadata={'source': 'AI 개론'}, page_content='자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'), Document(metadata={'source': '딥러닝 입문'}, page_content='컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.')]
벡터 저장소에 성공적으로 추가된 문서 개수 : 5
['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


In [21]:
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n 쿼리 : {query}")
print("가장 유사한 문서 : ")

for doc in results:
	print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")


 쿼리 : 인공지능과 머신러닝의 관계는?
가장 유사한 문서 : 
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI 개론]
- 딥러닝은 머신러닝의 한 종류입니다. [출처: 딥러닝 입문]


In [22]:
# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_5', 'DOC_3', 'DOC_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'AI 개론'},
  {'source': 'AI 개론'},
  {'source': '딥러닝 입문'},
  {'source': '딥러닝 입문'},
  {'source': 'AI 개론'}]}

### 문서 수정
- vector_store.update_document(document_id, document)

In [23]:
from langchain_core.documents import Document
# 업데이트할 문서 생성
updated_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.",
    metadata={"source": "AI 개론"},
)

updated_document_2 = Document(
    page_content="머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.",
    metadata={"source": "AI 개론"},
)

updated_document_3 = Document(
    page_content="딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.",
    metadata={"source": "딥러닝 입문"},
)

# 단일 문서 업데이트
chroma_db.update_document(document_id="DOC_1", document=updated_document_1)


In [24]:
# 여러 문서 한번에 업데이트
chroma_db.update_documents(
	ids=["DOC_2", "DOC_3"],
	documents=[updated_document_2, updated_document_3]
)

In [11]:
# 저장된 문서 검색 예시
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n 쿼리 : {query}")
print("가장 유사한 문서 : ")

for doc in results:
	print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")


 쿼리 : 인공지능과 머신러닝의 관계는?
가장 유사한 문서 : 
- 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다. [출처: AI 개론]
- 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다. [출처: AI 개론]


- 문서 삭제 : vector_store.delete(ids)

In [10]:
# 문서 id를 지정하여 삭제
chroma_db.delete(ids=["DOC_4"])

In [12]:
# 컬렉션 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_5', 'DOC_3'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.',
  '머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.',
  '딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'AI 개론'},
  {'source': 'AI 개론'},
  {'source': '딥러닝 입문'},
  {'source': '딥러닝 입문'}]}

### 유사도 점수 해석
- 유사도 점수가 포함된 검색
	- 유사도 점수를 함께 반환
	- 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)
- 검색 방법 비교
	- 매서드 : similarity_search
		- 반환값 : 문서 리스트만
		- 점수 해석 : 점수 없음
	- 매서드 : similarity_search_with_score
		- 반환값 : 문서, 거리점수
		- 점수 범위 : 0 - 무한대
		- 점수 해석 : 낮을수록 유사
	- 매서드 : similarity_search_with_relevance_scores
		- 반환값 : 문서, 관련성점수
		- 점수 범위 : 0 - 1
		- 점수 해석 : 높을수록 유사

In [13]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db.similarity_search_with_score(
	query,
	k=2,
	filter={"source": "AI 개론"}
)

print("점수가 포함된 유사도 검색 결과: \n")
for doc, score in results:
	print(f"- 점수: {score:.4f}")
	print(f"  내용: {doc.page_content}")
	print(f"  [출처: {doc.metadata['source']}]")
	print()

점수가 포함된 유사도 검색 결과: 

- 점수: 0.9258
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI 개론]

- 점수: 1.0149
  내용: 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.
  [출처: AI 개론]



- 관련성 점수가 포함된 검색
	- `similarity_search_with_relevance_scores` 사용
	- 문서와 함께 0에서 1 사이의 관련성 점수를 반환
	- 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [14]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db.similarity_search_with_relevance_scores(
	query,
	k=2,
	filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n 검색 결과 (관련성 점수 포함) : ")
for doc, score in results:
	print(f"- 점수: {score:.4f}")
	print(f"  내용: {doc.page_content}")
	print(f"  [출처: {doc.metadata['source']}]")
	print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

 검색 결과 (관련성 점수 포함) : 
- 점수: 0.3453
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI 개론]

- 점수: 0.2824
  내용: 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.
  [출처: AI 개론]



(4) 벡터 저장소 로드

In [15]:
chroma_db2 = Chroma(
	collection_name="ai_sample_collection",
	embedding_function=embeddings_model,
	persist_directory="../chroma_db"
)

In [16]:
# 미리 임베딩된 쿼리 백터를 사용하여 검색
query = "딥러닝은 어떤 분야에서 사용되나요?"

results = chroma_db2.similarity_search_with_relevance_scores(
	query,
	k=2,
	filter={"source": "딥러닝 입문"}
)

print(f"쿼리: {query}")
print("\n 검색 결과 (관련성 점수 포함) : ")
for doc, score in results:
	print(f"- 점수: {score:.4f}")
	print(f"  내용: {doc.page_content}")
	print(f"  [출처: {doc.metadata['source']}]")
	print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

 검색 결과 (관련성 점수 포함) : 
- 점수: 0.3987
  내용: 딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.
  [출처: 딥러닝 입문]

- 점수: 0.0926
  내용: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
  [출처: 딥러닝 입문]



In [17]:
# 컬렉션 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_5', 'DOC_3'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.',
  '머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.',
  '딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'AI 개론'},
  {'source': 'AI 개론'},
  {'source': '딥러닝 입문'},
  {'source': '딥러닝 입문'}]}

### 2. FAISS(Facebook AI Similarity Search)
- 효율적인 벡터 유사도 검색 및 클러스터링을 위한 오픈소스 벡터 저장소
- faiss-cpu 패키지 설치